In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("proj2.ipynb")

# Project 2: Query Performance
## Due Date: Thursday, October 5th, 5:00 PM

## Assignment Details
In this project, we will explore how the database system optimizes query execution and how users can futher tune the performance of their queries.

This project works with the Lahman's Baseball Database, an open source collection of baseball statistics from 1871 to 2020. It contains a variety of data, like batting statistics, team stats, managerial records, Hall of Fame records, and much more.

You may find this project's queries to be simpler than in Project 1. However, although the queries may not be as complex, we still expect you to spend ample time thinking through the effects of each of the methods, as reasoning about the tradeoff between different approaches is the goal of this assignment.

**Note:** If at any point during the project, the internal state of the database or its tables have been modified in an undesirable way (i.e. a modification not resulting from the instructions of a question), restart your kernel and clear output and simply re-run the notebook as normal. This will shutdown you current connection to the database, which will prevent the issue of multiple connections to the database at any given point, and when re-running the notebook you will create a fresh database based on the provided Postgres dump.

## Logistics & Scoring Breakdown

- Each coding question has **both public tests and hidden tests**. Roughly 50% of your coding grade will be made up of your score on the public tests released to you, while the remaining 50% will be made up of unreleased hidden tests.
- Public tests for multiple choice questions are for sanity check only (e.g. you are answering in the correct format). Partial credit will be awarded.
- Free-response questions will be manually graded. Please answer thoughtfully and concisely in complete sentences, drawing from knowledge in lectures and from your inspection of query plans.

This is an **individual project**. However, you’re welcome to collaborate with any other student in the class as long as it’s within the academic honesty guidelines.


| Question    | 0 | 1 | 2              | 3    | 4    | 5              | 6              | 7    | 8              | 9        | 10        |
| ----------- | - | - | -------------- | ---- | ---- | -------------- | -------------- | ---- | -------------- | -------- | --------- |
| No Subparts | 1 |   |                |      |      |                |                |      |                |          | 6         |
| a           |   | 1 | 1              | 1    | 1    | 2              | 2              | 2    | 1              | 1        |           |
| b           |   | 3 | 3              | 1    | 1    | 1              | 2              | 1    | 1              | 1        |           |
| c           |   |   | 1              | m: 2 | 1    | 1              | 1              | m: 3 | 1              | 2 (m: 2) |           |
| d           |   |   | 4 (m: 2, a: 2) | m: 2 | m: 3 | 4 (m: 2, a: 2) | 1              |      | 4 (m: 2, a: 2) |          |           |
| e           |   |   |                |      |      |                | 4 (m: 2, a: 2) |      |                |          |           |
| **Total**       | 1 | 4 | 9              | 6    | 6    | 8              | 10             | 6    | 7              | 4        | manual: 6 |


**Grand Total:** 67 points (manual: 26, autograded: 41)

In [ ]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

## Getting Connected
Similar to Project 1, we will be using the `JupySQL` library to connect this notebook to a PostgreSQL database server on your JupyterHub account. Run the following cell to initiate the connection.

In [ ]:
%reload_ext sql
%sql postgresql://jovyan@127.0.0.1:5432/postgres

In [ ]:
# See full display
%config SqlMagic.displaylimit = None

## Setting up the Database
The following cell will create the `baseball` database (if needed), unzip the Postgres dump of the Lahman's Baseball Database, populate the `baseball` database with the desired tables and data, and finally display all databases associated with the Postgres instance. After running the cell, you should see the `baseball` database in the generated list of databases outputted by `%sql \l`.

**Note:** If you run into the **role does not exist**/**database does not exist** error the first time you run this cell, feel free to ignore it. It does not affect data import.

In [ ]:
!unzip -u data/baseball.zip -d data/

In [ ]:
!psql postgresql://jovyan@127.0.0.1:5432/baseball -c 'SELECT pg_terminate_backend(pg_stat_activity.pid) FROM pg_stat_activity WHERE datname = current_database()  AND pid <> pg_backend_pid();'
!psql -h localhost -c 'DROP DATABASE IF EXISTS baseball'
!psql -h localhost -c 'CREATE DATABASE baseball'
!psql -h localhost -d baseball -f data/baseball.sql
!psql -h localhost -c 'SET max_parallel_workers_per_gather = 0;'
%sql \l

Now, run the following cell to connect to the `baseball` database. There should be no errors after running the following cell.

In [ ]:
%sql postgresql://jovyan@127.0.0.1:5432/baseball

To ensure that the connection to the database has been established, let's try grabbing the first 5 rows from the `halloffame` table.

In [ ]:
%%sql
SELECT * FROM halloffame LIMIT 5

## Connect to the grader

Run the following cell for grading purposes.

In [ ]:
# Just run the following cell, no further action is needed.
from data101_utils import GradingUtil
grading_util = GradingUtil("proj2")
grading_util.prepare_autograder()

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


## Table Descriptions
In its entirety the Lahman's Baseball Database contains 27 tables containing a variety of statistics for players, teams, games, schools, etc. For simplicity, this project will focus on a subset of the tables:
* `appearances`: details on the positions each player appeared at
* `batting`: batting statistics for each player
* `collegeplaying`: list of players and the colleges they attended
* `halloffame`: Hall of Fame voting data
* `people`: player information (name, date of birth, and biographical info)
* `salaries`: player salary data
* `schools`: list of colleges that players attended

As a reminder from Project 1, `%sql \d <table_name>` is helpful for identifying the columns in a table.

<br><br>
<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Question 0: PostgreSQL Explain Analyze
**Please read through this section carefully, as a vast majority of the project will require you to inspect query plans via interpreting the output of the explain analyze command.**

To inspect the query plan for a given query, create a variable storing the query as a string and invoke a `psql` shell command to `explain analyze` the query: 

`your_query_str = "__REPLACE_ME_WITH_QUERY__"`

`!psql -h localhost -d baseball -c "explain analyze $your_query_str"`

Take a look at the following sample query plan.

![title](data/sample_query.png)

It is highly recommended to read through [this article](https://www.cybertec-postgresql.com/en/how-to-interpret-postgresql-explain-analyze-output/) and the postgreSQL [documentation 14.1.2](https://www.postgresql.org/docs/current/using-explain.html#USING-EXPLAIN-ANALYZE) to see how you can interpret the output above. Everything before "Tools to interpret Explain Analyze output" is useful.


<div class="alert alert-block alert-info">
Here are some key things to note for all question parts:
<ul>
<li>When we ask you to identify the <b>query cost</b>, we are looking for the <b>total cost</b>.</li>
    <ul>
    <li>There are two cost values: the first is the <b>startup cost</b> (cost to return the first row) and the second is the <b>total cost</b> (cost to return all rows).</li>
    <li>The unit for the estimated query cost is an arbitrary estimation of disk I/O (1 is the cost for reading an 8kB page during a sequential scan).</li>
        <li>Feel free to round the query cost / time to the nearest integer, but we'll accept anything more exact.</li>
    </ul>
<li>When we ask you to identify the <b>query time</b>, we are looking for the <b>execution time</b> (in ms).</li>
    <ul>
        <li>We recognize that the execution time may vary between different cell executions, so the autograder will tolerate a reasonable range.</li>
    </ul>
</ul>
</div>

Now, inspect the query plan above by following the below steps:

1. Manually copy the entire query command (i.e., `SELECT ... `) from the screenshot into the cell below.

_Type your answer here, replacing this text._

In [ ]:
%%sql --save query_0 result_0 <<
...

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_0 = %sqlcmd snippets query_0
grading_util.save_results("result_0", query_0, result_0);
result_0.DataFrame().head(3)

In [ ]:
%sql EXPLAIN ANALYZE {{query_0}}

In [ ]:
sample_query_cost = ...
sample_query_timing = ...

In [ ]:
grader.check("q0")

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Question 1: Queries and Views, Part 1

In Questions 1 and 2, you will compare and contrast writing queries with subqueries and views.

## Question 1a
Write a query that finds `namefirst`, `namelast`, `playerid` and `yearid` of all people who were successfully inducted into the Hall of Fame. **Note**: Your query should **NOT** use any sub-queries.

In [ ]:
%%sql --save query_1a result_1a <<
...

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_1a = %sqlcmd snippets query_1a
grading_util.save_results("result_1a", query_1a, result_1a)
result_1a.DataFrame().head(3)

In [ ]:
grader.check("q1a")

<br><br>

---

## Question 1b
In this question, we will compare the query you wrote in `Question 1a` against the provided query below in `Question 1bi` by inspecting both query plans.

#### Question 1bi: 
Inspect the query plan for `provided_query` and the query you wrote in `Question 1a` by running the cells below.

In [ ]:
%%sql --save provided_query result_provided <<
-- just run this cell
SELECT namefirst, namelast, p.playerid, yearid
FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof 
WHERE p.playerid = hof.playerid;

In [ ]:
# just run this cell 
provided_query = %sqlcmd snippets provided_query
%sql EXPLAIN ANALYZE {{provided_query.strip(';')}}

In [ ]:
# just run this cell
%sql EXPLAIN ANALYZE {{query_1a}}

Record the **execution time** and **cost** for each query.

In [ ]:
provided_query_cost = ...
provided_query_timing = ...
your_query_cost = ...
your_query_timing = ...

In [ ]:
grader.check("q1bi")


#### Question 1bii:
Given your findings from inspecting the query plans of the two queries, answer the following question. Assign the variable `q1b_part2` to a list of all of the below statements that are true.


Consider the following statements:
<br>
A. Both the queries have the same cost
<br>
B. The provided query has a faster execution time because it makes use of a subquery.
<br>
C. The query you wrote has a faster execution time because it does not make use a subquery.
<br>
D. The provided query has less cost because it makes use of a subquery.
<br>
E. The query you wrote has less cost because it does not make use a subquery.
<br>
F. The queries have the same output.
<br>
G. The queries do not have the same output.
    
**Note:** Your answer should look like `q1b_part2 = ['A', 'B']`

In [ ]:
q1b_part2 = ...

In [ ]:
grader.check("q1bii")

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


# Question 2: Queries and Views, Part 2

In this question, you will continue analyzing queries with/without views and materialized views.
* Question 2a: Write a query that computes people in a Hall of Fame.
* Question 2 Tutorial: Use this query to create a view called `inducted_hof_ca` and a materialized view, `inducted_hof_ca_mat`.
* Question 2b: Write three queries that achieve the same result:
  * Question 2bi: One that uses no views.
  * Question 2bii: One that uses the `inducted_hof_ca` view.
  * Question 2ciii: One that uses the `inducted_hof_ca_mat` materialized view.
* Question 2c: Record the performance of these three queries.
* Question 2d: Analyze and discuss using queries with different types of views.

<br/><br/>

---

## Question 2a

Write a query that returns the people who were successfully inducted into the Hall of Fame and played in college at a school located in California. For each player, return their `namefirst`, `namelast`, `playerid`, `schoolid`, and `yearid` ordered by the `yearid` and then the `playerid`. 

**Note**: For this query, `yearid` refers to player's year of induction into the Hall of Fame.

In [ ]:
%%sql --save query_2a result_2a <<
...
;

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_2a = %sqlcmd snippets query_2a
grading_util.save_results("result_2a", query_2a, result_2a)
result_2a

In [ ]:
grader.check("q2a")

<br/><br/>

---

## Question 2 Tutorial

We are now going to use the query you wrote in the previous part to generate a view, called `inducted_hof_ca`, and a materialized view, `inducted_hof_ca_mat`.

Run the below cells. You do not need to do anything more for this part. 

(Note: the semicolon strip is to avoid executing an empty query with double-semicolons, which causes an error.)

In [ ]:
%%sql
/* just run this cell */
DROP VIEW IF EXISTS inducted_hof_ca;
CREATE VIEW inducted_hof_ca AS {{query_2a.strip(';')}};
SELECT * FROM inducted_hof_ca;

In [ ]:
%%sql
/* just run this cell */
DROP MATERIALIZED VIEW IF EXISTS inducted_hof_ca_mat;
CREATE MATERIALIZED VIEW inducted_hof_ca_mat AS {{query_2a.strip(';')}};
SELECT * FROM inducted_hof_ca_mat;

<br/><br/>

---
### Question 2b

For this question, we want to compute the count of players who were inducted into the Hall of Fame and played baseball at a college in California for each `schoolid` and `yearid` combination ordered by ascending `yearid`.

You should write three queries that accomplish this task, but with different strategies:
* Question 2bi: Use the `inducted_hof_ca` view;
* Question 2bii Use the `inducted_hof_ca_mat` view; and
* Question 2biii: Do not use `inducted_hof_ca` view, `inducted_hof_ca_mat` materialized view, any common table expressions (CTEs), nor any subqueries.

### Question 2bi

Write a query to accomplish the task above using the `inducted_hof_ca` view. Assign your result to `result_2b_view`.

In [ ]:
%%sql --save query_2b_view result_2b_view <<
...

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_2b_view = %sqlcmd snippets query_2b_view
grading_util.save_results("result_2b_view", query_2b_view, result_2b_view)
result_2b_view

In [ ]:
grader.check("q2bi")

<br/><br/>

#### Question 2bii:

Now, write the query a second time to use the materialized view `inducted_hof_ca_mat`. Assign your result to `result_2b_mat`.

In [ ]:
%%sql --save query_2b_mat result_2b_mat <<
...

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_2b_mat = %sqlcmd snippets query_2b_mat
grading_util.save_results("result_2b_mat", query_2b_mat, result_2b_mat)
result_2b_mat

In [ ]:
grader.check("q2bii")

<br/><br/>

#### Question 2biii:

Finally, write the query a third time to **not** use the `inducted_hof_ca` view, nor the `inducted_hof_ca_mat` materialized view, nor any common table expressions (CTEs), nor any subqueries. Save your result in `result_2b_no_view`.

In [ ]:
%%sql --save query_2b_no_view result_2b_no_view <<
...

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_2b_no_view = %sqlcmd snippets query_2b_no_view
grading_util.save_results("result_2b_no_view", query_2b_no_view, result_2b_no_view)
result_2b_no_view

In [ ]:
grader.check("q2biii")

<br/><br/>

---

### Question 2c
Inspect the query plans for the three queries you wrote above by running the following cells.

In [ ]:
# just run this cell
%sql EXPLAIN ANALYZE {{query_2b_view}}

In [ ]:
# just run this cell
%sql EXPLAIN ANALYZE {{query_2b_mat}}

In [ ]:
# just run this cell
%sql EXPLAIN ANALYZE {{query_2b_no_view}}

Then, record the execution time and cost for each query.

In [ ]:
with_view_cost = ...
with_view_timing = ...
with_materialized_view_cost = ...
with_materialized_view_timing = ...
without_view_cost = ...
without_view_timing = ...

In [ ]:
grader.check("q2c")

<br/><br/>

---

## Question 2d

Given your findings from inspecting the query plans in this Question, as well as your understanding of views and materialized views from lectures, discuss the takeaways of using views and materialized views.

### Question 2di

Assign the variable `q2di` to a list of all of the below statements that are true.

A. Views will reduce the execution time and the cost of a query.<br/>
B. Views will reduce the execution time of a query, but not the cost.<br/>
C. Views will reduce the cost of a query, but not the execution time.<br/>
D. Materialized views reduce the execution time and the cost of a query.<br/>
E. Materialized views reduce the execution time, but not cost of a query<br/>
F. Materialized views reduce the cost of a query, but not the execution time.<br/>
G. Materialized views will result in the same query plan as a query using views.<br/>
H. Materialized views and views take the same time to create.<br/>
I. Materialized views take less time to create than a view.<br/>
J. Materialized views take more time to create than a view.<br/>
    
*Note:* Your answer should look like `q2di = ['A', 'B']`

In [ ]:
q2di = ...

In [ ]:
grader.check("q2di")

<!-- BEGIN QUESTION -->

#### Question 2dii:

Explain your answer to the previous part (Question 2di) based on your knowledge from lectures and details from the query plans. Your explanation should also include why you didn't choose certain options. Please answer in maximum 5 sentences.

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<br><br>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Question 3: Predicate Pushdown
In this question, we will explore the impact of predicates (i.e., filters) on a query's execution, particularly inspecting when the optimizer applies predicates.

* Question 3a: Compute a query with all rows.
* Question 3b: Add a simple filter.
* Question 3c: Analyze the tradeoffs to cost.
* Question 3d: Analyze the tradeoffs to execution time.


## Question 3a:
Recall the `inducted_hof_ca` view created in `Question 2`. Inspect the query plan for a query that that gets all rows from the view, and record the execution time and cost using an `EXPLAIN ANALYZE` command.

In [ ]:
%%sql
...

In [ ]:
query_view_cost = ...
query_view_timing = ...

In [ ]:
grader.check("q3a")

<br><br>

---

## Question 3b:
Now, add a filter to only return rows from `inducted_hof_ca` where the year is later than 2010. Inspect the query plan and record the execution time and cost.

In [ ]:
%%sql
...

In [ ]:
query_view_with_filter_cost = ...
query_view_with_filter_timing = ...

In [ ]:
grader.check("q3b")

<!-- BEGIN QUESTION -->

## Question 3c:
Given your findings from inspecting the query plans of queries from Questions 3a and 3b, fill in the blank and **justify your answer**. Explain your answer based on your knowledge from lectures, and details from the query plans (your explanation should include why you didn't choose other options). Your response should be no longer than 3 sentences.

**Note:** Your answer should be formatted as follows: `A because ...`

**Adding a filter ___ the cost.**
<br>
A. increased
<br>
B. decreased
<br>
C. did not change

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<br><br>

---

## Question 3d:
Given your findings from inspecting the query plans of queries from Questions 3a and 3b, fill in the blank and **justify your answer**. Explain your answer based on your knowledge from lectures, and details from the query plans (your explanation should include why you didn't choose other options). Your response should be no longer than 3 sentences.

**Note:** Your answer should be formatted as follows: `A because ...`

**Adding a filter ___ the execution time.**
<br>
A. increased
<br>
B. decreased
<br>
C. did not change

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<br><br>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Question 4: Join Approaches

In this question, we'll explore different join approaches (Nested Loop Join, Merge Join, Hash Join) and discuss how the query optimizer picks the best approach.

<br/><br/>

---

## Question 4a
Perform an inner join on the `people` and `collegeplaying` tables on the `playerid` column. Project all columns.

In [ ]:
%%sql --save query_4a result_4a <<
...

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_4a = %sqlcmd snippets query_4a
grading_util.save_results("result_4a", query_4a, result_4a);

display(result_4a.DataFrame().head(3))
%sql EXPLAIN ANALYZE {{query_4a}} 

Run the cell above to inspect the query plan for your command.

**Which join approach did the query optimizer choose?** 

A. Nested Loop Join<br/>
B. Merge Join<br/>
C. Hash Join<br/>
D. None of the Above

Assign the variable `q4a` to the correct letter choice above, e.g., `q4a = 'A'`.

In [ ]:
q4a = ...

In [ ]:
grader.check("q4a")

<br><br>

---

## Question 4b

Similar to Question 4a, perform an inner join on the `people` and `collegeplaying` tables on the `playerid` column. Project all columns.

In addition, **sort your output by `playerid`.**

In [ ]:
%%sql --save query_4b result_4b <<
...

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_4b = %sqlcmd snippets query_4b
grading_util.save_results("result_4b", query_4b, result_4b);

display(result_4b.DataFrame().head(3))
%sql EXPLAIN ANALYZE {{query_4b}} 

Run the cell above to inspect the query plan for your command.

**Which join approach did the query optimizer choose?** 

A. Nested Loop Join<br/>
B. Merge Join<br/>
C. Hash Join<br/>
D. None of the Above

Assign the variable `q4b` to the correct letter choice above, e.g., `q4b = 'A'`.

In [ ]:
q4b = ...

In [ ]:
grader.check("q4b")

<br><br>

---
## Question 4c
Write a query to retrieve all possible player pair combinations. Project all columns, but **limit to 1000 rows** to ensure your query doesn't take an exorbitant amount of time to run.

**Hint:** You can do this by performing an inner join of the `people` table on itself with an inequality condition.

In [ ]:
%%sql --save query_4c result_4c <<
...

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_4c = %sqlcmd snippets query_4c
grading_util.save_results("result_4c", query_4c, result_4c);

display(result_4c.DataFrame().head(3))
%sql EXPLAIN ANALYZE {{query_4c}} 

Run the cell above to inspect the query plan for your command.

**Which join approach did the query optimizer choose?** 

A. Nested Loop Join<br/>
B. Merge Join<br/>
C. Hash Join<br/>
D. None of the Above

Assign the variable `q4c` to the correct letter choice above, e.g., `q4c = 'A'`.

In [ ]:
q4c = ...

In [ ]:
grader.check("q4c")

<!-- BEGIN QUESTION -->

<br><br>

---
## Question 4d

Given your findings above, why did the query optimizer ultimately choose the specific join approach you found in each of the above three scenarios in Questions 4a, 4b, and 4c? Feel free to discuss the pros and cons of each join approach as well.

If you feel stuck, here are some things to consider: Does a non-equijoin constrain us to certain join approaches? What's an added benefit in regards to the output of merge join?

**Note:** Your answer should be formatted as follows: `Q4a: A because ... Q4b: A because ...` You should write no more than 5 sentences.

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Question 5: Indexes, Part 1

In Questions 5, 6, and 7, you will analyze how indexes impact query performance.

Question 5:
* Question 5a: Write a query.
* Question 5b: Add an index with a particular index key and reanalyze the previous query's performance.
* Question 5c: Add an index with a different key and reanalyze the previous query's performance.

<br/>

---

## Question 5a
Write a query that outputs the `playerid` and average `salary` for each player that only batted in 10 games (the number of games in which a player batted can be found in the `g_batting` column of the `appearances` table). Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [ ]:
%%sql --save query_5a result_5a <<
...

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_5a = %sqlcmd snippets query_5a
grading_util.save_results("result_5a", query_5a, result_5a);

display(result_5a.DataFrame().head(3))
%sql EXPLAIN ANALYZE {{query_5a}} 

Inspect the query plan above and record the execution time and cost.

In [ ]:
result_5a_cost = ...
result_5a_timing = ...

In [ ]:
grader.check("q5a")

<br><br>

---
## Question 5b

Add an index with name `g_batting_idx` on the `g_batting` column of the `appearances` table.

In [ ]:
%%sql
DROP INDEX IF EXISTS g_batting_idx;
...

Now, re-inspect the query plan of the query from `Question 5a` and record its execution time and cost.

In [ ]:
# just run this cell
%sql EXPLAIN ANALYZE {{query_5a}} 

In [ ]:
result_5b_cost = ...
result_5b_timing = ...

In [ ]:
grader.check("q5b")

In the following question, we will explore adding a different index and evaluating the query from `Question 4a`. To avoid any interference by the `g_batting_idx` index, **drop the index before moving onto the next question.**

In [ ]:
# you must run this cell!!!
%sql DROP INDEX IF EXISTS g_batting_idx;

<br><br>

---
## Question 5c

Write a query to add an index with name `salary_idx` on the `salary` column of the `salaries` table. Make sure to drop the previous index in `Question 5b` first!

In [ ]:
%%sql
DROP INDEX IF EXISTS g_batting_idx;
DROP INDEX IF EXISTS salary_idx;
...

Now, re-inspect the query plan of the query from `Question 5a` and record its execution time and cost.

In [ ]:
# just run this cell
%sql EXPLAIN ANALYZE {{query_5a}} 

In [ ]:
result_5c_cost = ...
result_5c_timing = ...

In [ ]:
grader.check("q5c")

<br><br>

---

## Question 5d

Given your findings from inspecting the query plans with no indexes (Question 5a), an index on `g_batting` (Question 5b), and an index on `salary` (Question 5c), assign the variable `q5d` to a list of all of the below statements that are true.

A. Adding the `g_batting` index did not have a significant impact on the query execution time and cost.<br/>
B. Adding the `g_batting` index did have a significant impact on the query execution time, but not the cost.<br/>
C. Adding the `g_batting` index did have a significant impact on the query cost, but not the execution time.<br/>
D. Adding the `g_batting` index did have a significant impact on the query cost and execution time.<br/>
E. Adding the `salary` index did not have a significant impact on the query execution time and cost.<br/>
F. Adding the `salary` index did have a significant impact on the query execution time, but not the cost.<br/>
G. Adding the `salary` index did have a significant impact on the query cost, but not the execution time.<br/>
H. Adding the `salary` index did have a significant impact on the query cost and execution time.

**Note:** Your answer should be formatted as a list of single-character strings, e.g., `q5d = ['A', 'B']`

In [ ]:
q5d = ...

In [ ]:
grader.check("q5d")

<!-- BEGIN QUESTION -->

### Question 5di Justification

Explain your answer to `Question 5d` above based on your knowledge from lectures, and details from inspecting the query plans (your explanation should include why you didn't choose certain options). Your answer should be no longer than 3 sentences.

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Question 6: Indexes, Part 2

Continue the analysis on how indexes impact query performance.

Question 6:
* Question 6a: Write a query that uses an **and** boolean operator. Record query performance.
* Question 6b: Write a query that uses an **or** boolean operator. Record query performances.
* Question 6c: Add an index and rerun queries in Questions 6a, 6b. Record query performance.
* Question 6d: Add a multi-attribute index and rerun queries 6a, 6b. Record query performance.
* Question 6e: Analyze query performance; compare and contrast.

Before continuing, make sure to drop all existing indexes from previous questions.

In [ ]:
# you must run this cell!!!
%sql DROP INDEX IF EXISTS g_batting_idx;
%sql DROP INDEX IF EXISTS salary_idx;

<br><br>

---

## Question 6a

Write a query that finds the `playerid`, `yearid`, and `salary` for each player that had played 10 games **and** batted in 10 games (the number of games in which a player played can be found in the `g_all` column of the `appearances` table). Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [ ]:
%%sql --save query_6a result_6a <<
...

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_6a = %sqlcmd snippets query_6a
grading_util.save_results("result_6a", query_6a, result_6a);

result_6a.DataFrame().head(3)

In [ ]:
grader.check("q6a")

Inspect the query plan and record the execution time and cost.

In [ ]:
%sql EXPLAIN ANALYZE {{query_6a}} 

In [ ]:
result_6a_cost = ...
result_6a_timing = ...

In [ ]:
grader.check("6a_cost")

## Question 6b
Write a query that finds the `playerid`, `yearid`, and `salary` for each player that had played 10 games __or__ batted in 10 games. Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [ ]:
%%sql --save query_6b result_6b <<
...

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_6b = %sqlcmd snippets query_6b
grading_util.save_results("result_6b", query_6b, result_6b);
result_6b.DataFrame().head(3)

In [ ]:
grader.check("q6b")

Inspect the query plan and record the execution time and cost.

In [ ]:
%sql EXPLAIN ANALYZE {{query_6b}} 

In [ ]:
result_6b_cost = ...
result_6b_timing = ...

In [ ]:
grader.check("6b_cost")

## Question 6c
Now, let's see the impact of adding an index on the `g_batting` column. Create an index on the `g_batting` column. Re-inspect the queries from `Question 6a` and `Question 6b` and record the respective execution costs and times.

In [ ]:
%%sql
DROP INDEX IF EXISTS g_batting_idx;
...

In [ ]:
# record the updated costs for Question 6a ("and" query)
%sql EXPLAIN ANALYZE {{query_6a}} 

In [ ]:
result_6cand_index_cost = ...
result_6cand_index_timing = ...

In [ ]:
# record the updated costs for Question 6b ("or" query)
%sql EXPLAIN ANALYZE {{query_6b}} 

In [ ]:
result_6cor_index_cost = ...
result_6cor_index_timing = ...

In [ ]:
grader.check("q6c")

<br/><br/>

---

## Question 6d: Multiple-attribute index

Now, create a multiple column index on `g_batting` and `g_all` called `g_batting_g_all_idx` and record the query execution time and cost for the "or" command in `Question 6b`.

Before continuing, make sure to drop all existing indexes from previous questions.

In [ ]:
# you must run this cell!!!
%sql DROP INDEX IF EXISTS g_batting_idx;
%sql DROP INDEX IF EXISTS salary_idx;

In [ ]:
%%sql 
DROP INDEX IF EXISTS g_batting_all_idx;
...

In [ ]:
# record the updated costs for Question 6b ("or" query)
%sql EXPLAIN ANALYZE {{query_6b}} 

In [ ]:
result_6d_multiple_col_index_cost = ...
result_6d_multiple_col_index_timing = ...

In [ ]:
grader.check("q6d")

<br/><br/>

---

## Question 6e
Given your findings from inspecting the query plans from all parts of this `Question 6`, assign the variable `q6e` to a list of all below statements that are true.

A. Adding an index on a column used in an AND predicate will reduce the query time, but not the execution cost.<br/>
B. Adding an index on a column used in an AND predicate will reduce the query cost, but not the execution time.<br/>
C. Adding an index on a column used in an AND predicate will reduce the query cost and the execution time.<br/>
D. Adding an index on a column used in an OR predicate will reduce the query time, but not the execution cost.<br/>
E. Adding an index on a column used in an OR predicate will reduce the query cost, but not the execution time.<br/>
F. Adding an index on a column used in an OR predicate will reduce the query cost and the execution time.<br/>
G. Adding a multicolumn index on columns in an OR predicate will reduce the query time, but not the execution cost.<br/>
H. Adding a multicolumn index on columns in an OR predicate will reduce the query cost, but not the execution time.<br/>
I. Adding a multicolumn index on columns in an OR predicate will reduce the query cost and the execution time.

**Note:** Your answer should be formatted as a list of single-character strings, e.g., `q6e = ['A', 'B']`


In [ ]:
q6e = ...

In [ ]:
grader.check("q6e")

<!-- BEGIN QUESTION -->

### Question 6ei Justification

Explain your answer to `Question 6e` above based on your knowledge from lectures, and details from inspecting the query plans (your explanation should include why you didn't choose certain options). Your answer should be no longer than 3 sentences.

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Question 7: Indexes, Part 3

Continue the analysis on how indexes impact query performance. Now, use aggregators.

Question 7:
* Question 7a: Write two queries that use aggregators. Record query performance.
* Question 7b: Add an index and rerun queries in Questions 7a. Record query performance.
* Question 7c: Analyze query performance; compare and contrast.

Before continuing, make sure to drop all existing indexes from previous questions.

In [ ]:
# you must run this cell!!!
%sql DROP INDEX IF EXISTS g_batting_idx;
%sql DROP INDEX IF EXISTS salary_idx;
%sql DROP INDEX IF EXISTS g_batting_all_idx;

---

## Question 7a

Write 2 queries: one that finds the minimum salary from the salary table `Salaries` and one that finds the average. Inspect the queries' query plans and record their execution times and costs.

Minimum salary:

In [ ]:
%%sql --save query_7a_min result_7a_min << 
...

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_7a_min = %sqlcmd snippets query_7a_min
grading_util.save_results("result_7a_min", query_7a_min, result_7a_min);

display(result_7a_min)
%sql EXPLAIN ANALYZE {{query_7a_min}} 

In [ ]:
result_7a_min_query_cost = ...
result_7a_min_query_timing = ...

In [ ]:
grader.check("q7a_min")

Average salary:

In [ ]:
%%sql --save query_7a_avg result_7a_avg <<
...

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_7a_avg = %sqlcmd snippets query_7a_avg
grading_util.save_results("result_7a_avg", query_7a_avg, result_7a_avg);

display(result_7a_avg)
%sql EXPLAIN ANALYZE {{query_7a_avg}} 

In [ ]:
result_7a_avg_query_cost = ...
result_7a_avg_query_timing = ...

In [ ]:
grader.check("q7a_avg")

<br><br>

---
## Question 7b
Create an index on the `salary` column in the `Salaries` table and re-inspect the query plans from the previous part and record the respective execution time and cost.

In [ ]:
%%sql 
DROP INDEX IF EXISTS salary_idx;
...

In [ ]:
# record the updated costs for "min" query
%sql EXPLAIN ANALYZE {{query_7a_min}} 

In [ ]:
result_7b_min_query_cost = ...
result_7b_min_query_timing = ...

In [ ]:
# record the updated costs for "avg" query
%sql EXPLAIN ANALYZE {{query_7a_avg}} 

In [ ]:
result_7b_avg_query_cost = ...
result_7b_avg_query_timing = ...

In [ ]:
grader.check("q7b")

<!-- BEGIN QUESTION -->

<br><br>

---

## Question 7c
Given your findings from `Question 7`, which of the following statements is true?
<br> A. An index on the column being aggregated in a query will always provide a performance enhancement.
<br> B. A query finding the MIN(salary) will always benefit from an index on salary, but a query finding MAX(salary) will not.
<br> C. A query finding the COUNT(salary) will always benefit from an index on salary, but a query finding AVG(salary) will not.
<br> D. Queries finding the MIN(salary) or MAX(salary) will always benefit from an index on salary, but queries finding AVG(salary) or COUNT(salary) will not.

**Justify your answer.** Explain your answer based on your knowledge from lectures, and details of the query plans (your explanation should include why you didn't choose certain options). Your response should be no longer than 3 sentences.
 
*Note:* Your answer should be formatted as follows: "A because ... " 

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Question 8: Clustered Indexes
In this question, we will inspect the impact that clustering our data on an index can have on a query's performance.
* Question 8a: query
* Question 8b: cluster index on primary key
* Question 8c: cluster index on different key
* Question 8d: observe and analyze.

Before continuing, make sure to drop all existing indexes from previous questions.

In [ ]:
# you must run this cell!!!
%sql DROP INDEX IF EXISTS g_batting_idx;
%sql DROP INDEX IF EXISTS salary_idx;
%sql DROP INDEX IF EXISTS g_batting_all_idx;

---

## Question 8a

Write a query that finds the `playerid`, `yearid`, `teamid`, and `ab` for all players whose `ab` was above 500. Inspect the query plan and record the execution time and cost.

In [ ]:
%%sql --save query_8a result_8a <<
...

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_8a = %sqlcmd snippets query_8a
grading_util.save_results("result_8a", query_8a, result_8a);
result_8a.DataFrame().head(3)

In [ ]:
grader.check("q8a")

Inspect the query plan and record the execution time and cost.

In [ ]:
%sql EXPLAIN ANALYZE {{query_8a}} 

In [ ]:
result_8a_cost = ...
result_8a_timing = ...

In [ ]:
grader.check("8a_cost")

<br><br>

---

## Question 8b

Cluster the `batting` table on its primary key (hint: use the psql meta-command `\di` to find out what name of the primary key is). We are able to directly cluster on the primary key (without first creating a separate index) because Postgres automatically creates an index for it.

Then, re-inspect the query plan for the query from `Question 8a` and record the execution time and cost.

In [ ]:
%%sql
...

In [ ]:
# check the updated costs for query in Question 8a
%sql EXPLAIN ANALYZE {{query_8a}} 

In [ ]:
result_8b_cost = ...
result_8b_timing = ...

In [ ]:
grader.check("q8b")

<br><br>

---

## Question 8c

Now, let's try clustering the table based on another index. Create an index on `ab` column called `ab_idx` in the `batting` table AND cluster the `batting` table with this new index. Now, re-inspect the query plan and record the execution time and cost.

In [ ]:
%%sql --save query_8c result_8c <<
...

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_8c = %sqlcmd snippets query_8c
grading_util.save_results("result_8c", query_8c, result_8c);

# check the updated costs for query in Question 8a
%sql EXPLAIN ANALYZE {{query_8a}} 

In [ ]:
result_8c_cost = ...
result_8c_timing = ...

In [ ]:
grader.check("q8c")

<br><br>

---

## Question 8d
Given your findings from inspecting the query plans from Questions 8a, 8b, and 8c, assign the variable `q8d` to a list of all statements that are true.

A. Clustering based on the `ab_idx` decreased the cost of the query.<br/>
B. Clustering based on the `ab_idx` increased the cost of the query.<br/>
C. Clustering based on the `ab_idx` increased the execution time of the query.<br/>
D. Clustering based on the `ab_idx` decreased the execution time of the query.<br/>
E. Clustering based on the `batting_pkey` decreased the cost of the query.<br/>
F. Clustering based on the `batting_pkey` increased the cost of the query.<br/>
G. Clustering based on the `batting_pkey` increased the execution time of the query.<br/>
H. Clustering based on the `batting_pkey` decreased the execution time of the query.<br/>
I. None of the above
    
**Note:** Your answer should be formatted as a list of single-character strings, e.g., `q8d = ['A', 'B']`.


In [ ]:
q8d = ...

In [ ]:
grader.check("q8d")

<br><br>

---

### Question 8di Justification

Explain your answer to `Question 8d` above based on your knowledge from lectures, and details from inspecting the query plans (your explanation should include why you didn't choose certain options). Your answer should be no longer than 3 sentences.

_Type your answer here, replacing this text._

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Question 9: Cost of Index Management
Until now, we have seen the positive potential impact that indexes can have on query performance, but remember in real world technologies/applications, we will be routinely receiving new data (and in large quantities) which would trigger regular updates to our tables. In this section, we will dive into the cost of managing the indexes that we create.

Before starting this question, be sure to delete any indexes by running the below cell.

In [ ]:
# you must run this cell!!!
%sql DROP INDEX IF EXISTS g_batting_idx;
%sql DROP INDEX IF EXISTS salary_idx;
%sql DROP INDEX IF EXISTS g_batting_all_idx;
%sql DROP INDEX IF EXISTS ab_idx;

---

## Question 9a

Record the time it takes to insert 300,000 rows into the `salaries` table when no additional index is configured.

Run the following cell to setup a column to track which rows we added as part of these inserts.

In [ ]:
%sql ALTER TABLE salaries ADD added boolean DEFAULT False;

Next, run the provided update script and record the **wall time**.

**NOTE:** Running the below cell multiple times may result in an error, unless you first delete the rows with the cell given at the end of this subpart.

In [ ]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100001..400000 LOOP
     INSERT INTO salaries (yearid, teamid, lgid, playerid, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

In [ ]:
result_9a_timing = ...

In [ ]:
grader.check("q9a")

<br/><br/>

**Before moving onto the next question**,  delete all the rows that were added to the table from the update script.

In [ ]:
%%sql
/* just run this cell */
DELETE FROM salaries
WHERE added = 'true';

<br><br>

---

## Question 9b

Now, create an index on the `salary` column and record the **wall time** after executing the update script. Make sure to first run the previous cell to rollback any changes from the previous part!

In [ ]:
%%sql 
...

**NOTE:** Running the below cell multiple times may result in an error, unless you first delete the rows with the cell given at the end of last subpart.

In [ ]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100001..400000 LOOP
     INSERT INTO salaries (yearid, teamid, lgid, playerid, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

In [ ]:
result_9b_timing = ...

In [ ]:
grader.check("q9b")

<!-- BEGIN QUESTION -->

<br><br>

---

## Question 9c:
What difference did you notice when you added an index into the salaries table and re-timed the update? Why do you think it happened? Your answer should be no longer than 3 sentences.

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Question 10: Project Takeaways

In this project, we explored how the database system optimizes query execution and how users can futher tune the performance of their queries.

Familiarizing yourself with these optimization and tuning methods will make you a better data engineer. In this question, we'll ask you to recall and summarize these concepts. Who knows? Maybe one day it will help you during an interview or on a project.

In the following answer cell,
1. Name 3 methods you learned in this project. The method can be either the optimization done by the database system, or the fine tuning done by the user.
2. For each method, summarize how and why it can optimize query performance. Feel free to discuss any drawbacks, if applicable.

Your answer should be no longer than ten sentences. Each method identification/discussion is 2 points.


_Type your answer here, replacing this text._

<!-- END QUESTION -->

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Congratulations! You have finished Project 2.

Run the following cell to zip and download the results of your queries. You will also need to run the export cell at the end of the notebook.

**Please save your notebook before exporting (this is a good time to do it!)** Otherwise, we may not be able to register your written responses.

**For submission on Gradescope, you will need to submit BOTH the `proj2.zip` file generated by the export cell and the `results.zip` file generated by the following cell.**

**Common submission issues:** You MUST submit the generated zip files (not folders) to the autograder. However, Safari is known to automatically unzip files upon downloading. You can fix this by going into Safari preferences, and deselect the box with the text "Open safe files after downloading" under the "General" tab. If you experience issues with downloading via clicking on the link, you can also navigate to the project 2 directory within JupyterHub (remove `proj2.ipynb` from the url), and manually download the generated zip files. Please post on Ed if you encounter any other submission issues.

In [ ]:
grading_util.prepare_submission_and_cleanup()  # builds results.zip

In [ ]:
# Close SQL magic connection
# You may disregard "RunTimeError: Could not close connection"
# %sql --close postgresql://127.0.0.1:5432/baseball

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True, files=['results.zip'])